# Connect to bio.tools API

These West-Life services are processed here:
- DisVis web portal
- PowerFit web portal
- HADDOCK web portal
- Virtual Folder for Structural Biology Projects
- PDB_REDO server 
- The Protein Crystallisation Construct Designer
- Scipion
- 3DBIONOTES
- ARP wARP
- CS-ROSETTA3
- 'FANTEN'
- 'SpotOn'
- 'Dipcheck'  
- 'AMBER'
- 'MetalPDB' 

TODO PDB_REDO server

These West-Life servies are not in bio.tools and so are not processed:
- xplor-nih
- ccp4online
- autorickshaw
- vici


In [120]:
LOGOS = {
    #'DisVis web portal': , no logo
    #'PowerFit web portal': , no logo
    'HADDOCK web portal': 'http://haddock.science.uu.nl/enmr/services/HADDOCK2.2/HADDOCK-logo.png' ,
    #'Virtual Folder for Structural Biology Projects': ,
    'PDB_REDO server': 'https://pdb-redo.eu/images/PDB_logo_rect_medium.svg',
    #'Protein Crystallisation Construct Designer': ,
    'Scipion': 'http://scipionwebtools.westlife.fedcloud.eu/m/resources/scipion_logo_normal.png', 
    '3DBIONOTES': 'https://3dbionotes.cnb.csic.es/assets/3dbionotes_ws_logo-6652d19a5137adc910eb93da6d77658e451ddeae1318b01eb313b271f3e92c0f.png', 
    'ARP wARP': 'https://arpwarp.embl-hamburg.de/static/arp.png',   
    #'CS-ROSETTA3',  no logo,
    'MetalPDB': 'http://metalweb.cerm.unifi.it/global/images/MetalPDB.png',
    'Dipcheck': 'http://cluster.embl-hamburg.de/dipcheck/images/dipcheck.png'
    
}


## Get West-Life tools

In [121]:
# 
import requests

# a GET request
response = requests.get('https://dev.bio.tools/api/tool/?collectionID=West-Life&format=json',
    # could  "Accept": "application/json;charset=UTF-8"
)
assert 200 == response.status_code
btservices = response.json()['list']
'loaded'

'loaded'

## Convert to einfracentral format

In [131]:
import json

def convert(btservice):
    eiservice = {
        # required terms 
        'name': btservice['name'],
        'url': btservice['homepage'],
        'order': btservice['download'][0]['url'] if btservice['download'] else btservice['homepage'], 
        "providers": ['West-Life'],

        # Required, but no good match in bio.tools
        'symbol': ( LOGOS[btservice['name']] if btservice['name'] in LOGOS
                   else 'https://about.west-life.eu/network/west-life/templates/westlife/images/west-life.png'), 
        "category": "Category-Software", "subcategory": "Subcategory-Tool",
        "places": ["Place-WW"],
        
        # length limit in einfacentral
        'funding' : "EC funding ",
        
        # terms that map fairly well
        # could register all 'providers': btservice['collectionID'], # nearest approximation
        'version': btservice['version'][0] if btservice['version'] else '1.0',
        "lastUpdate": btservice['lastUpdate'],
        'description': btservice['description'],
        'tags': [ t['term'] for t in btservice['topic']],
        # btservice['language'] is computer language of implementation, not natural language of UI
        
        # defaults, revised below if possible
        "serviceLevelAgreement": "https://auth.west-life.eu/aai/AUP.pdf", 
        'helpdesk': 'https://about.west-life.eu/network/west-life/documentation',
        'options': '',
        "termsOfUse": ["https://auth.west-life.eu/aai/AUP.pdf"], 
        "languages": ["Language-en"],

    }
    if btservice['license']:
        if btservice['license'].startswith('http'):
            eiservice['termsOfUse'] = [btservice['license']]
        elif ( 'GNU General Public License v2' == btservice['license'].strip()
            or 'GPL-2.0'  == btservice['license'].strip() ):
            eiservice['termsOfUse'] = ['https://www.gnu.org/licenses/gpl-2.0.html']
        else:
            print('licence', '<'+btservice['license']+'>')
    for f in btservice['function']:
        for op in f['operation']:
            eiservice['options'] = eiservice['options'] +op['term']+', '
    if btservice['function']:
        for f in btservice['function']:
            if 'note' in f and f['note']:
                tagline = f['note']
                eiservice['tagline'] = tagline if len(tagline)<100 else tagline[0:97]+'...'
                break;
    if 'Mature'==btservice['maturity'] or btservice['maturity'] is None:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Production"
        eiservice["trl"]= "TRL-9"
    elif 'Emerging'==btservice['maturity']:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Beta"
        eiservice["trl"]= "TRL-7" # permissible levels are 9,8,7
    else:
        print('maturity', btservice['maturity'])
        
    for d  in btservice['documentation']:
        if 'Citation instructions' == d['type']:
            pass
        elif 'Terms of Use' == d['type']:
            eiservice['termsOfUse'] = [d['url']]
            eiservice['serviceLevelAgreement'] = d['url']
    for d in btservice['credit']:
        # unfortunately "max length for 'funding' is 500 chars"
        #if  'Funding agency' == d['typeEntity']:
            # eiservice['funding'] = eiservice['funding']+', '+d['name']
        if d['typeRole'] and 'Primary Contact' in d['typeRole']:
            eiservice['helpdesk'] = d['url']
    return eiservice

# options, 
eiservices = []
for btservice in btservices:
    eiservice = convert(btservice)
    # validate it        
    HEADERS = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    response = requests.post('http://beta.einfracentral.eu/api/service/validate', 
        data=json.dumps(eiservice), headers=HEADERS)

    assert 200 == response.status_code, response.json()
    eiservices.append(eiservice)
    print(eiservice['name'])
'validated'

DisVis web portal
PowerFit web portal
HADDOCK web portal
Virtual Folder for Structural Biology Projects
PDB_REDO server
Protein Crystallisation Construct Designer


'validated'

## Now handle services not listed as West-Life

In [132]:
SERVICES = [
    'DisVis', 'PowerFit web portal','HADDOCK',
    'Virtual Folder for Structural Biology Projects',
    'FANTEN',
    'SpotOn', 'Dipcheck', 'AMBER', 'MetalPDB',
    'Scipion', '3DBIONOTES', 'ARP wARP',   'CS-ROSETTA3',  
    'The Protein Crystallisation Construct Designer',
    #'PDB_REDO server'
    # 'DisVis web portal', 'HADDOCK web portal',
]
for service in SERVICES:
    response = requests.get('https://bio.tools/api/tool/?name='+service
        # could  "Accept": "application/json;charset=UTF-8"
    )
    assert 200 == response.status_code
    if 0==len(response.json()['list']):
        print('Not found:', service)
    for btservice in response.json()['list']:
        if service!=btservice['name']:
            print('Spurious service found:', btservice['name'], 'for:', service )
            continue
        eiservice = convert(btservice)
        # validate it        
        HEADERS = {'Content-Type': 'application/json', 'Accept': 'application/json'}
        response = requests.post('http://beta.einfracentral.eu/api/service/validate', 
            data=json.dumps(eiservice), headers=HEADERS)

        assert 200 == response.status_code, response.json()
        eiservices.append(eiservice)
        print(eiservice['name'])
'validated'  

DisVis
PowerFit web portal
Spurious service found: HDOCK for: HADDOCK
HADDOCK
Virtual Folder for Structural Biology Projects
FANTEN
Spurious service found: FANTOM for: FANTEN
Spurious service found: VANTED for: FANTEN
SpotOn
Dipcheck
AMBER
MetalPDB
Scipion
3DBIONOTES
ARP wARP
CS-ROSETTA3
The Protein Crystallisation Construct Designer
Not found: PDB_REDO server


'validated'